In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
import os
import logging
from tqdm import tqdm
import json

# ログの設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('/workspaces/mal_gpu/logs/tokenizer_training_wordpiece.log'),  # ログをファイルに出力
        logging.StreamHandler()  # 標準出力にも出力
    ]
)

# 入力ファイルのパス
concatenated_file = '/workspaces/mal_gpu/data/interim/tokenizer_training/tokenizer_training_data_concatenated01.txt'
non_concatenated_file = '/workspaces/mal_gpu/data/interim/tokenizer_training/tokenizer_training_data_non_concatenated01.txt'

# ファイルの存在チェック
if not os.path.exists(concatenated_file):
    logging.error(f"ファイルが見つかりません: {concatenated_file}")
    exit(1)

if not os.path.exists(non_concatenated_file):
    logging.error(f"ファイルが見つかりません: {non_concatenated_file}")
    exit(1)

# トークナイザーの訓練関数
def train_wordpiece_tokenizer(input_file, output_tokenizer_dir, is_concatenated=True):
    """WordPieceトークナイザーを訓練"""
    logging.info(f"WordPieceトークナイザーの訓練を開始します。入力ファイル: {input_file}")

    # WordPieceモデルの作成
    tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
    trainer = trainers.WordPieceTrainer(
        vocab_size=50000,  # 語彙サイズは必要に応じて調整
        min_frequency=2,
        special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "memoryaddress", "immediatevalue"]
    )

    # 正規化器と前処理器の設定
    tokenizer.normalizer = normalizers.BertNormalizer(
        clean_text=True,
        handle_chinese_chars=True,
        strip_accents=False,
        lowercase=True  # 必要に応じてFalseに変更
    )
    tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

    # ファイル内の総行数をカウント（進捗バーのため）
    total_lines = sum(1 for _ in open(input_file, 'r', encoding='utf-8'))

    # 大容量ファイルをストリーム処理するためのジェネレーター関数
    def batch_iterator(batch_size=1000):
        with open(input_file, 'r', encoding='utf-8') as f:
            batch = []
            for line in tqdm(f, total=total_lines, desc="WordPieceトークナイザーの訓練中", ncols=100):
                line = line.strip()
                if line == '':
                    continue
                batch.append(line)
                if len(batch) >= batch_size:
                    yield batch
                    batch = []
            if batch:
                yield batch

    # トークナイザーの訓練
    tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

    # 出力ディレクトリが存在しない場合、作成する
    if not os.path.exists(output_tokenizer_dir):
        os.makedirs(output_tokenizer_dir)

    # トークナイザーの保存 (vocab.txt を保存)
    tokenizer.model.save(output_tokenizer_dir, prefix='vocab')

    # Tokenizer config を保存
    tokenizer_config = {
        "model_type": "bert",
        "tokenizer_class": "BertTokenizer",
        "do_lower_case": True,  # 必要に応じて変更
        "unk_token": "[UNK]",
        "sep_token": "[SEP]",
        "pad_token": "[PAD]",
        "cls_token": "[CLS]",
        "mask_token": "[MASK]",
        "special_tokens": ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "memoryaddress", "immediatevalue"]
    }
    with open(os.path.join(output_tokenizer_dir, "tokenizer_config.json"), "w") as f:
        json.dump(tokenizer_config, f)

    logging.info(f"WordPieceトークナイザーの訓練が完了しました。保存先: {output_tokenizer_dir}")

# WordPieceトークナイザーの訓練実行（連結パターン）
train_wordpiece_tokenizer(
    input_file=concatenated_file,
    output_tokenizer_dir='/workspaces/mal_gpu/data/interim/tokenizer_training/wordpiece/concatenated_tokenizer',
    is_concatenated=True  # 連結パターン
)

# 必要に応じて、非連結データでも訓練を行う場合は以下を実行
# WordPieceトークナイザーの訓練実行（非連結パターン）
# train_wordpiece_tokenizer(
#     input_file=non_concatenated_file,
#     output_tokenizer_dir='/workspaces/mal_gpu/data/interim/tokenizer_training/wordpiece/non_concatenated_tokenizer',
#     is_concatenated=False  # 非連結パターン
# )
